In [1]:
%load_ext ipycache

/home/zulissi/miniconda3/envs/cgcnn/lib/python3.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/home/zulissi/miniconda3/envs/cgcnn/lib/python3.7/site-packages/ipycache.py:17: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils.traitlets import Unicode


In [2]:
import os
import sys
# sys.path.insert(0,'/home/junwoony/.local/lib/python3.6/site-packages')
# sys.path.insert(0,'/home/zulissi/software/adamwr')
import numpy as np
#Select which GPU to use if necessary
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0
%env CUDA_LAUNCH_BLOCKING=1
import mongo
# from torchviz import make_dot, make_dot_from_trace
import time
from torch.nn.utils import clip_grad_norm_
import pickle
import random
from torch.utils.data import Dataset, DataLoader
import mongo
from cgcnn.data_grad import StructureData, ListDataset, StructureDataTransformer
import numpy as np
import tqdm
from sklearn.preprocessing import StandardScaler

import multiprocess as mp
from sklearn.model_selection import ShuffleSplit

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0
env: CUDA_LAUNCH_BLOCKING=1


In [ ]:
%%cache cached_docs.pkl docs SDT_list  orig_atom_fea_len nbr_fea_len
docs = pickle.load(open('./input/H_docs.pkl', 'rb'))

docs = [doc for doc in docs if doc['coordination']==doc['initial_coordination'] and doc['coordination']!=''
                               and doc['movement_data']['max_adsorbate_movement']<1.0
                               and doc['movement_data']['max_slab_movement']<1.0
                               and 'N' not in doc['atoms']['chemical_symbols'] #no nitrides
                               and 'C' not in doc['atoms']['chemical_symbols'] #no carbides
                               and 'S' not in doc['atoms']['chemical_symbols'] #no sulfides
                               and 'Se' not in doc['atoms']['chemical_symbols'] #no selenides
                               and doc['atoms']['symbol_counts']['H']<2 #no hydrides
        ]
random.shuffle(docs)
docs = docs[0:10000]

start = time.time()

SDT = StructureDataTransformer(atom_init_loc='atom_init.json',
                              max_num_nbr=12,
                               step= 0.2,
                              radius=12,
                              use_tag=True,
                              use_fixed_info=True,
                              bond_property=True,
#                               use_voronoi=False,
#                               train_geometry = 'initial',
                              is_initial=True
                              )

SDT_out = SDT.transform(docs)

structures = SDT_out[0]

#Settings necessary to build the model (since they are size of vectors as inputs)
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]


SDT_out = SDT.transform(docs)

with mp.Pool(4) as pool:
#     SDT_list = list(tqdm.tqdm(map(lambda x: SDT_out[x],range(len(SDT_out))),total=len(SDT_out)))
    SDT_list = list(tqdm.tqdm(pool.imap(lambda x: SDT_out[x],range(len(SDT_out)),chunksize=40),total=len(SDT_out)))

structures = SDT_list[0]
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]


end = time.time()
SDT_time = end-start
print('SDT conversion time', SDT_time)
print('SDT list created')

In [ ]:
target_list = np.array([sdt[-1].numpy() for sdt in SDT_list]).reshape(-1,1)
# pickle.dump(SDT_list, open('./input/SDT_20000_grad.pkl', 'wb'))


In [ ]:
from torch.optim import Adam, SGD
from sklearn.model_selection import ShuffleSplit
from skorch.callbacks import Checkpoint, LoadInitState #needs skorch 0.4.0, conda-forge version at 0.3.0 doesn't cut it

from cgcnn.data_grad import collate_pool, MergeDataset
from cgcnn.model_grad_2 import CrystalGraphConvNet
from skorch import NeuralNetRegressor
import torch
import skorch.callbacks.base


cuda = torch.cuda.is_available()
if cuda:
    device = torch.device("cuda")
else:
    device='cpu'

#Make a checkpoint to save parameters every time there is a new best for validation lost
cp = Checkpoint(monitor='valid_loss_best',fn_prefix='valid_best_')

#Callback to load the checkpoint with the best validation loss at the end of training

class train_end_load_best_valid_loss(skorch.callbacks.base.Callback):
    def on_train_end(self, net, X, y):
        net.load_params('valid_best_params.pt')
        
load_best_valid_loss = train_end_load_best_valid_loss()
print('device', device)

In [ ]:
from sklearn.model_selection import train_test_split

SDT_list= SDT_list
target_list = target_list

indices = np.arange(len(SDT_list))
SDT_training, SDT_test, target_training, target_test, train_idx, test_idx \
= train_test_split(SDT_list, target_list, indices, test_size=0.2, random_state=42)

# Benchmark for the validation set

We want to get an idea for what the DFT initial - final structure difference is; the ML models needs to beat this number for the validation set. 

In [ ]:
def diff(target,sdt):
    free_atom_idx = list(set(list(range(len(target[0]))))-set(sdt[-2].numpy()))
    diff = np.sum(((target[0] - sdt[4].numpy())[free_atom_idx])**2.,axis=1)**0.5
    return diff

np.mean(np.abs(np.concatenate([diff(target,sdt) for sdt,target in zip(SDT_test, target_test)])))

In [ ]:
from skorch.dataset import CVSplit
from skorch.callbacks.lr_scheduler import WarmRestartLR, LRScheduler
from adamwr.adamw import AdamW
from torch.optim.lbfgs import LBFGS

from adamwr.cosine_scheduler import CosineLRWithRestarts
from sklearn.model_selection import train_test_split

train_test_splitter = ShuffleSplit(test_size=0.25, random_state=42)

# warm restart scheduling from https://arxiv.org/pdf/1711.05101.pdf
LR_schedule = LRScheduler(CosineLRWithRestarts, batch_size=100, epoch_size=len(SDT_training), restart_period=10, t_mult=1.2)

#############
# To extract intermediate features, set the forward takes only the first return value to calculate loss
class MyNet(NeuralNetRegressor):
    def get_loss(self, y_pred, y_true, **kwargs):        
        y_pred = y_pred[0] if isinstance(y_pred, tuple) else y_pred  # discard the 2nd output
        return torch.mean(torch.sqrt(torch.sum((y_pred-y_true.cuda())**2.0,dim=1)))
#         return super().get_loss(y_pred, y_true, **kwargs)
## return features = net.forward(SDT_test)
############

net = MyNet(
    CrystalGraphConvNet,
    module__orig_atom_fea_len = orig_atom_fea_len,
    module__nbr_fea_len = nbr_fea_len,
    batch_size=100, #214
    module__classification=False,
    lr=0.0056,
    max_epochs= 1000, 
    module__atom_fea_len=46, #46,
    module__h_fea_len=83,
    module__n_conv=8, #8
    module__n_h=8,
        module__opt_step_size=0.3,
    module__min_opt_steps=20,
    module__max_opt_steps=300,
    module__steps=4,
    optimizer__weight_decay=1e-3,
    optimizer=AdamW,
    iterator_train__pin_memory=True,
    iterator_train__num_workers=0,
    iterator_train__shuffle=True,
    iterator_train__collate_fn = collate_pool,
    iterator_valid__pin_memory=True,
    iterator_valid__num_workers=0,
    iterator_valid__collate_fn = collate_pool,
    device=device,
#     criterion=torch.nn.MSELoss,
    criterion=torch.nn.L1Loss,
    dataset=MergeDataset,
    train_split = CVSplit(cv=train_test_splitter),
    callbacks=[cp, LR_schedule, load_best_valid_loss] #    callbacks=[cp, load_best_valid_loss, LR_schedule]

)

In [ ]:
net.initialize()
net.module_.max_opt_steps=300
net.module_.max_epochs=1000
net.partial_fit(SDT_training, target_training)
# net.module_.min_opt_steps=300
# net.module_.max_epochs=10
# net.partial_fit(SDT_training, target_training)


In [ ]:
import ase.io
mean_error_ml = []
mean_error_dft = []

with ase.io.trajectory.TrajectoryWriter('test.traj') as traj:
    for i, (doc, sdt) in enumerate(zip(np.array(docs)[test_idx], SDT_test)):
        atoms_final = mongo.make_atoms_from_doc(doc)
        final_pos = sdt[-1]
        atoms_final.positions=final_pos
        atoms_init = mongo.make_atoms_from_doc(doc['initial_configuration'])
        ml_pos = list(net.forward_iter([sdt]))[0][-1].detach().numpy()
        atoms_ml = atoms_final.copy()
        atoms_ml.positions = ml_pos
        traj.write(atoms_init)
        traj.write(atoms_ml)
        traj.write(atoms_final)
        diff = np.abs(np.sqrt(np.sum((ml_pos-final_pos.numpy())**2.,axis=1)))
        diff = diff[diff>1e-6]
        
        mean_error_ml.append(np.mean(np.abs(diff)))
        
        diff = np.abs(np.sqrt(np.sum((atoms_init.positions-final_pos.numpy())**2.,axis=1)))
        diff = diff[diff>1e-6]
        mean_error_dft.append(np.abs(diff))

mean_error_ml = np.array(mean_error_ml)

In [ ]:
np.mean(mean_error_ml)